In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/MT_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4814097007,MT,"100 Consumer Direct Way, MISSOULA, MT",46.905731,-114.030968,3.006300e+14
1,4358347005,MT,"13 N 23RD ST, BILLINGS, MT",45.784929,-108.498976,3.011100e+14
2,7436157007,MT,"3424 US HWY 12E, HELENA, MT",46.592415,-111.965265,3.004900e+14
3,4750337203,MT,"216 14TH AVE SW, SIDNEY, MT",47.715075,-104.173240,3.008307e+14
4,7853207108,MT,"175 North 27th Street Suite 800, BILLINGS, MT",45.783424,-108.504401,3.011100e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4814097007,MT,"100 Consumer Direct Way, MISSOULA, MT",46.905731,-114.030968,3.006300e+14
1,4358347005,MT,"13 N 23RD ST, BILLINGS, MT",45.784929,-108.498976,3.011100e+14
2,7436157007,MT,"3424 US HWY 12E, HELENA, MT",46.592415,-111.965265,3.004900e+14
3,4750337203,MT,"216 14TH AVE SW, SIDNEY, MT",47.715075,-104.173240,3.008307e+14
4,7853207108,MT,"175 North 27th Street Suite 800, BILLINGS, MT",45.783424,-108.504401,3.011100e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
200,7468168403,MT,"303 Cass Ave, Fairview, MT",47.854173,-104.036959,NaN
271,4489387006,MT,"808 MAIN ST, LAUREL, MT",39.109217,-76.856913,NaN
1898,1119017304,MT,"US HIGHWAY 933253 US Hwy 93 N, STEVENSVILLE, MT",41.415801,-114.749709,NaN
1978,3510988507,MT,"6600 Highway 12 W, Lolo, MT",44.848031,-91.603171,NaN
2128,5657827001,MT,"16132 Hwy 200 E PO Box 487, FAIRVIEW, MT",47.862350,-104.040398,NaN
2513,1518898608,MT,"104 Central Ave, Hobson, MT",40.700379,-73.929576,NaN
2956,4369657010,MT,"6600 HIGHWAY 12 W, LOLO, MT",44.848031,-91.603171,NaN
2998,4365937003,MT,"4026 US Hwy 93 N, STEVENSVILLE, MT",41.415801,-114.749709,NaN
3006,2539967101,MT,"3201 US HWY 93 S, KALISPELL, MT",41.415801,-114.749709,NaN
3100,1698977804,MT,"741 Highway 212, RED LODGE, MT",44.934020,-95.732065,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('30')] #MT
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/10 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

10


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_13634/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
200,7468168403,MT,"303 Cass Ave, Fairview, MT",47.854173,-104.036959,[]
271,4489387006,MT,"808 MAIN ST, LAUREL, MT",39.109217,-76.856913,[]
1898,1119017304,MT,"US HIGHWAY 933253 US Hwy 93 N, STEVENSVILLE, MT",41.415801,-114.749709,[]
1978,3510988507,MT,"6600 Highway 12 W, Lolo, MT",44.848031,-91.603171,[]
2128,5657827001,MT,"16132 Hwy 200 E PO Box 487, FAIRVIEW, MT",47.862350,-104.040398,[]
2513,1518898608,MT,"104 Central Ave, Hobson, MT",40.700379,-73.929576,[]
2956,4369657010,MT,"6600 HIGHWAY 12 W, LOLO, MT",44.848031,-91.603171,[]
2998,4365937003,MT,"4026 US Hwy 93 N, STEVENSVILLE, MT",41.415801,-114.749709,[]
3006,2539967101,MT,"3201 US HWY 93 S, KALISPELL, MT",41.415801,-114.749709,[]
3100,1698977804,MT,"741 Highway 212, RED LODGE, MT",44.934020,-95.732065,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_13634/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
200,7468168403,MT,"303 Cass Ave, Fairview, MT",47.854173,-104.036959,NaN
271,4489387006,MT,"808 MAIN ST, LAUREL, MT",39.109217,-76.856913,NaN
1898,1119017304,MT,"US HIGHWAY 933253 US Hwy 93 N, STEVENSVILLE, MT",41.415801,-114.749709,NaN
1978,3510988507,MT,"6600 Highway 12 W, Lolo, MT",44.848031,-91.603171,NaN
2128,5657827001,MT,"16132 Hwy 200 E PO Box 487, FAIRVIEW, MT",47.862350,-104.040398,NaN
2513,1518898608,MT,"104 Central Ave, Hobson, MT",40.700379,-73.929576,NaN
2956,4369657010,MT,"6600 HIGHWAY 12 W, LOLO, MT",44.848031,-91.603171,NaN
2998,4365937003,MT,"4026 US Hwy 93 N, STEVENSVILLE, MT",41.415801,-114.749709,NaN
3006,2539967101,MT,"3201 US HWY 93 S, KALISPELL, MT",41.415801,-114.749709,NaN
3100,1698977804,MT,"741 Highway 212, RED LODGE, MT",44.934020,-95.732065,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
200,7468168403,MT,"303 Cass Ave, Fairview, MT",47.854173,-104.036959,NaN
271,4489387006,MT,"808 MAIN ST, LAUREL, MT",39.109217,-76.856913,NaN
1898,1119017304,MT,"US HIGHWAY 933253 US Hwy 93 N, STEVENSVILLE, MT",41.415801,-114.749709,NaN
1978,3510988507,MT,"6600 Highway 12 W, Lolo, MT",44.848031,-91.603171,NaN
2128,5657827001,MT,"16132 Hwy 200 E PO Box 487, FAIRVIEW, MT",47.862350,-104.040398,NaN
2513,1518898608,MT,"104 Central Ave, Hobson, MT",40.700379,-73.929576,NaN
2956,4369657010,MT,"6600 HIGHWAY 12 W, LOLO, MT",44.848031,-91.603171,NaN
2998,4365937003,MT,"4026 US Hwy 93 N, STEVENSVILLE, MT",41.415801,-114.749709,NaN
3006,2539967101,MT,"3201 US HWY 93 S, KALISPELL, MT",41.415801,-114.749709,NaN
3100,1698977804,MT,"741 Highway 212, RED LODGE, MT",44.934020,-95.732065,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
200,7468168403,NaN
271,4489387006,NaN
1898,1119017304,NaN
1978,3510988507,NaN
2128,5657827001,NaN
2513,1518898608,NaN
2956,4369657010,NaN
2998,4365937003,NaN
3006,2539967101,NaN
3100,1698977804,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/MT_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,518299,4814097007,MT,"100 Consumer Direct Way, MISSOULA, MT",46.905731,-114.030968,3.006300e+14
1,518300,4358347005,MT,"13 N 23RD ST, BILLINGS, MT",45.784929,-108.498976,3.011100e+14
2,518301,7436157007,MT,"3424 US HWY 12E, HELENA, MT",46.592415,-111.965265,3.004900e+14
3,518302,4750337203,MT,"216 14TH AVE SW, SIDNEY, MT",47.715075,-104.173240,3.008307e+14
4,518303,7853207108,MT,"175 North 27th Street Suite 800, BILLINGS, MT",45.783424,-108.504401,3.011100e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,518299,4814097007,MT,"100 Consumer Direct Way, MISSOULA, MT",46.905731,-114.030968,3.006300e+14,NaN
1,518300,4358347005,MT,"13 N 23RD ST, BILLINGS, MT",45.784929,-108.498976,3.011100e+14,NaN
2,518301,7436157007,MT,"3424 US HWY 12E, HELENA, MT",46.592415,-111.965265,3.004900e+14,NaN
3,518302,4750337203,MT,"216 14TH AVE SW, SIDNEY, MT",47.715075,-104.173240,3.008307e+14,NaN
4,518303,7853207108,MT,"175 North 27th Street Suite 800, BILLINGS, MT",45.783424,-108.504401,3.011100e+14,NaN
...,...,...,...,...,...,...,...,...
3137,521436,8146707002,MT,"2718 MONTANA AVE, BILLINGS, MT",45.781363,-108.504133,3.011100e+14,NaN
3138,521437,8283197000,MT,"PO BOX 5, FORSYTH, MT",46.263050,-106.681979,3.008700e+14,NaN
3139,521438,8286498608,MT,"909 Wyoming Ave, Billings, MT",45.778312,-108.539072,3.011100e+14,NaN
3140,521439,8968548509,MT,"10126 US Highway 2 E, Coram, MT",48.418884,-114.046013,3.002900e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,518299,4814097007,MT,"100 Consumer Direct Way, MISSOULA, MT",46.905731,-114.030968,3.006300e+14,NaN,3.006300e+14
1,518300,4358347005,MT,"13 N 23RD ST, BILLINGS, MT",45.784929,-108.498976,3.011100e+14,NaN,3.011100e+14
2,518301,7436157007,MT,"3424 US HWY 12E, HELENA, MT",46.592415,-111.965265,3.004900e+14,NaN,3.004900e+14
3,518302,4750337203,MT,"216 14TH AVE SW, SIDNEY, MT",47.715075,-104.173240,3.008307e+14,NaN,3.008307e+14
4,518303,7853207108,MT,"175 North 27th Street Suite 800, BILLINGS, MT",45.783424,-108.504401,3.011100e+14,NaN,3.011100e+14


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4814097007,MT,"100 Consumer Direct Way, MISSOULA, MT",46.905731,-114.030968,3.006300e+14
1,4358347005,MT,"13 N 23RD ST, BILLINGS, MT",45.784929,-108.498976,3.011100e+14
2,7436157007,MT,"3424 US HWY 12E, HELENA, MT",46.592415,-111.965265,3.004900e+14
3,4750337203,MT,"216 14TH AVE SW, SIDNEY, MT",47.715075,-104.173240,3.008307e+14
4,7853207108,MT,"175 North 27th Street Suite 800, BILLINGS, MT",45.783424,-108.504401,3.011100e+14
...,...,...,...,...,...,...
3137,8146707002,MT,"2718 MONTANA AVE, BILLINGS, MT",45.781363,-108.504133,3.011100e+14
3138,8283197000,MT,"PO BOX 5, FORSYTH, MT",46.263050,-106.681979,3.008700e+14
3139,8286498608,MT,"909 Wyoming Ave, Billings, MT",45.778312,-108.539072,3.011100e+14
3140,8968548509,MT,"10126 US Highway 2 E, Coram, MT",48.418884,-114.046013,3.002900e+14


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
200,7468168403,MT,"303 Cass Ave, Fairview, MT",47.854173,-104.036959,NaN
271,4489387006,MT,"808 MAIN ST, LAUREL, MT",39.109217,-76.856913,NaN
1898,1119017304,MT,"US HIGHWAY 933253 US Hwy 93 N, STEVENSVILLE, MT",41.415801,-114.749709,NaN
1978,3510988507,MT,"6600 Highway 12 W, Lolo, MT",44.848031,-91.603171,NaN
2128,5657827001,MT,"16132 Hwy 200 E PO Box 487, FAIRVIEW, MT",47.862350,-104.040398,NaN
2513,1518898608,MT,"104 Central Ave, Hobson, MT",40.700379,-73.929576,NaN
2956,4369657010,MT,"6600 HIGHWAY 12 W, LOLO, MT",44.848031,-91.603171,NaN
2998,4365937003,MT,"4026 US Hwy 93 N, STEVENSVILLE, MT",41.415801,-114.749709,NaN
3006,2539967101,MT,"3201 US HWY 93 S, KALISPELL, MT",41.415801,-114.749709,NaN
3100,1698977804,MT,"741 Highway 212, RED LODGE, MT",44.934020,-95.732065,NaN


Convert to File

In [19]:
fips_MT = fips_merge_drop

In [20]:
fips_MT.to_csv("../../../data/state_data/geo/geo_fips/23/MT_fips_scraped.csv") 